In [1]:
import pandas as pd 
import os 

In [4]:
RAW_DATA_PATH = "./ml-latest-small/"

In [5]:
ratings_csv_file = os.path.join(RAW_DATA_PATH, 'ratings.csv')
movies_csv_file = os.path.join(RAW_DATA_PATH, 'movies.csv')


ratings = pd.read_csv(ratings_csv_file)
movies = pd.read_csv(movies_csv_file)

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings.sort_values(by='timestamp', inplace=True)

In [13]:
def densify_index(df : pd.DataFrame):
    userMap = {v: i for i, v in enumerate(df['userId'].unique())}
    movieMap = {v: i for i, v in enumerate(df['movieId'].unique())}
    
    df['userId'] = df['userId'].map(userMap)
    df['movieId'] = df['movieId'].map(movieMap)

    return df, userMap, movieMap


densified_df, userMap, movieMap = densify_index(ratings)
densified_df.head()

,userId,movieId,rating,timestamp
66719,0,0,5.0,828124615
66716,0,1,5.0,828124615
66717,0,2,5.0,828124615
66718,0,3,5.0,828124615
66712,0,4,3.0,828124615


In [18]:
ratings_grpby_user = ratings.groupby(by='userId')
ratings_grpby_user.head()

,userId,movieId,rating,timestamp
66719,0,0,5.0,828124615
66716,0,1,5.0,828124615
66717,0,2,5.0,828124615
66718,0,3,5.0,828124615
66712,0,4,3.0,828124615
...,...,...,...,...
51200,609,112,5.0,1537157510
51203,609,616,5.0,1537157513
51197,609,114,5.0,1537157516
51209,609,1438,4.0,1537157521


In [49]:
grps = list(ratings_grpby_user.groups)

In [29]:
import random 

MASK = 1
# Input에 들어가기 전에 시퀀스를 마스킹한다.
def random_mask_itemList(input, mask_rate=.8):
    return [item if random.random() < mask_rate else MASK for item in input]

    # what if original item for mask 
    return [item if random.randint(2, totalItems)  ]
    

In [33]:
def get_context(df, context_size, val_context_size = 5):
    end_index = random.randint(10, df.shape[0] - val_context_size)
    start_index = max(0, end_index - context_size)
    context = df[start_index:end_index]
    return context

In [51]:
PAD = 0 
padding_mode = "left" if random.random() < .5 else "right"

def pad_list(item_list, history_size, mode, pad_val=PAD):
    current_items_count = len(item_list)
    padding_count = history_size - current_items_count
    if padding_count > 0:
        pads = [pad_val] * padding_count
        if mode == "left":
            item_list = pads + item_list 
        else:
            item_list = item_list + pads
    return item_list 


In [54]:
idx = 0 # userID
grp = grps[idx] # userID 0's grouped dataframe
df = ratings_grpby_user.get_group(grp) # get that grouped df
print(df.shape)
context = get_context(df, context_size=120) # generate model input
print(context.columns, context.shape)
target_items = context['movieId'].tolist()
source_items = random_mask_itemList(target_items, mask_rate=.8)
print(source_items) # 1's are masked items
print(target_items) 
padded_source_items = pad_list(source_items, 120, mode='left')
padded_target_items = pad_list(target_items, 120, mode='left')
print(padded_source_items)


(58, 4)
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object') (46, 4)
[0, 1, 2, 3, 4, 1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 1, 25, 26, 1, 28, 1, 30, 1, 32, 1, 34, 1, 1, 1, 38, 39, 40, 41, 42, 43, 44, 45]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 1, 25, 26, 1, 28, 1, 30, 1, 32, 1, 34, 1, 1, 1, 38, 39, 40, 41, 42, 43, 44, 45]
